### Гопиенко Александр Дмитриевич КИ22-17/2Б

# Практическая работа №4 по реккурентным нейронным сетям

In [52]:
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
from tqdm import tqdm 

In [53]:
ACTIVITIES = {
    1: 'WALKING',
    2: 'WALKING_UPSTAIRS',
    3: 'WALKING_DOWNSTAIRS',
    4: 'SITTING',
    5: 'STANDING',
    6: 'LAYING',
}

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [54]:
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI HAR dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        )

    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    filename = f'UCI HAR dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

def plot_train_val_accuracy_loss(history):
    plt.figure(figsize=(10, 10))

    plt.subplot(2, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    plt.subplot(2, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

In [55]:
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

X_train, X_test, Y_train, Y_test = load_data()
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

C:\Users\User\AppData\Local\Temp\ipykernel_13360\1713751055.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
C:\Users\User\AppData\Local\Temp\ipykernel_13360\1713751055.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
C:\Users\User\AppData\Local\Temp\ipykernel_13360\1713751055.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  return pd.read_csv(filename, delim_whitespace=True, header=None)
C:\Users\User\AppData\Local\Temp\ipykernel_13360\1713751055.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future ve

In [56]:
print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


### Датасет

In [57]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
train_dataset = CustomDataset(X_train, Y_train)
test_dataset = CustomDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# RNN

In [58]:
class RNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, n_classes):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]  
        out = self.fc(out)
        return out

# LSTM

In [ ]:
class LstmClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, n_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# GRU

In [ ]:
class GruClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, n_classes):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, n_classes)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Bidirectional RNN

In [ ]:
class BRnnClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, n_classes):
        super().__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, n_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [59]:
def train(model, optimizer, criterion, train_loader, device):
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        y = torch.argmax(y, dim=1)


        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(train_loader)

In [60]:
def evaluate(model, criterion, val_loader, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device)
            y = torch.argmax(y, dim=1)

            outputs = model(X)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return total_loss / len(val_loader), correct / total 

In [ ]:
def objective(trial, model_option="rnn"):
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_categorical('lr', [1e-4, 1e-3, 1e-2])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD'])

    if model_option == "rnn":
        model = RNNClassifier(input_dim, hidden_dim, num_layers, n_classes).to(device)
    elif model_option == "lstm":
        model = LstmClassifier(input_dim, hidden_dim, num_layers, n_classes).to(device)
    elif model_option == "gru":
        model = GruClassifier(input_dim, hidden_dim, num_layers, n_classes).to(device)
    elif model_option == "brnn":
        model = BRnnClassifier(input_dim, hidden_dim, num_layers, n_classes).to(device)

    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    criterion = nn.CrossEntropyLoss()

    best_val_loss = float('inf')
    patience = 5 
    counter = 0

    for epoch in tqdm(range(10), desc="Training"):  
        train_loss = train(model, optimizer, criterion, train_loader, device)
        val_loss, val_acc = evaluate(model, criterion, test_loader, device)

        trial.report(val_loss, epoch)  

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping on epoch {epoch}")
                break

        if trial.should_prune():  
            raise optuna.TrialPruned()

    return best_val_loss

[I 2025-04-26 18:31:41,900] A new study created in memory with name: no-name-a918b557-2520-4cc0-a5d0-5237bd9e44b3
Training: 100%|██████████| 10/10 [00:37<00:00,  3.77s/it]
[I 2025-04-26 18:32:19,568] Trial 0 finished with value: 1.203473840349464 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'lr': 0.0001, 'optimizer': 'Adam'}. Best is trial 0 with value: 1.203473840349464.
Training:  70%|███████   | 7/10 [01:26<00:37, 12.42s/it]
[I 2025-04-26 18:33:46,546] Trial 1 finished with value: 0.7647518818416903 and parameters: {'hidden_dim': 128, 'num_layers': 2, 'lr': 0.001, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7647518818416903.


Early stopping on epoch 7


Training: 100%|██████████| 10/10 [01:19<00:00,  7.96s/it]
[I 2025-04-26 18:35:06,177] Trial 2 finished with value: 1.7963588929945422 and parameters: {'hidden_dim': 64, 'num_layers': 2, 'lr': 0.0001, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.7647518818416903.
Training: 100%|██████████| 10/10 [01:19<00:00,  7.99s/it]
[I 2025-04-26 18:36:26,106] Trial 3 finished with value: 1.1074553410452541 and parameters: {'hidden_dim': 64, 'num_layers': 2, 'lr': 0.0001, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.7647518818416903.
Training: 100%|██████████| 10/10 [00:36<00:00,  3.69s/it]
[I 2025-04-26 18:37:03,058] Trial 4 finished with value: 1.454681348095658 and parameters: {'hidden_dim': 32, 'num_layers': 1, 'lr': 0.001, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.7647518818416903.


Best params: {'hidden_dim': 128, 'num_layers': 2, 'lr': 0.001, 'optimizer': 'Adam'}
Best score: 0.7647518818416903


# RNN

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, model_option="rnn")  

print("Best params:", study.best_params)
print("Best score:", study.best_value)

# LSTM

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, model_option="lstm")  

print("Best params:", study.best_params)
print("Best score:", study.best_value)

# GRU

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, model_option="gru")  

print("Best params:", study.best_params)
print("Best score:", study.best_value)

# Bidirectional RNN

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5, model_option="brnn")  

print("Best params:", study.best_params)
print("Best score:", study.best_value)